<a href="https://colab.research.google.com/github/loresiensis/data-analysis-and-nlp/blob/main/Crear-un-Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este caso usé un corpus EN-ES de https://www.statmt.org/europarl/ que consistía en un archivo comprimido con dos archivos de texto, uno en inglés y otro en español. Así que decidí subir ambos archivos de texto en lugar de descargar el dataset tal cual estaba en la web:

In [ ]:
with open('/content/europarl-en.txt') as f:
    text_en = f.readlines()

In [ ]:
text_en[:20]

['Resumption of the session\n',
 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.\n',
 "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.\n",
 'You have requested a debate on this subject in the course of the next few days, during this part-session.\n',
 "In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.\n",
 "Please rise, then, for this minute' s silence.\n",
 "(The House rose and observed a minute' s silence)\n",
 'Madam President, on a point of order.\n',
 'You will be aware from the press and television th

In [ ]:
with open('/content/europarl-es.txt') as f:
    text_es = f.readlines()

In [ ]:
text_es[:20]

['Reanudación del período de sesiones\n',
 'Declaro reanudado el período de sesiones del Parlamento Europeo, interrumpido el viernes 17 de diciembre pasado, y reitero a Sus Señorías mi deseo de que hayan tenido unas buenas vacaciones.\n',
 'Como todos han podido comprobar, el gran "efecto del año 2000" no se ha producido. En cambio, los ciudadanos de varios de nuestros países han sido víctimas de catástrofes naturales verdaderamente terribles.\n',
 'Sus Señorías han solicitado un debate sobre el tema para los próximos días, en el curso de este período de sesiones.\n',
 'A la espera de que se produzca, de acuerdo con muchos colegas que me lo han pedido, pido que hagamos un minuto de silencio en memoria de todas las víctimas de las tormentas, en los distintos países de la Unión Europea afectados.\n',
 'Invito a todos a que nos pongamos de pie para guardar un minuto de silencio.\n',
 '(El Parlamento, de pie, guarda un minuto de silencio)\n',
 'Señora Presidenta, una cuestión de procedimie

Como en este caso tengo dos archivos txt que ya están alineados, uso la función zip para emparejarlos

In [ ]:
aligned_texts = list(zip(text_en,text_es))

Construyo un fichero csv para guardar esta información:


In [ ]:
import pandas as pd
df = pd.DataFrame(aligned_texts,columns=['EN','ES'])
df.to_csv('texts_en_es.csv',index=None)
df

,EN,ES
0,Resumption of the session\n,Reanudación del período de sesiones\n
1,I declare resumed the session of the European ...,Declaro reanudado el período de sesiones del P...
2,"Although, as you will have seen, the dreaded '...","Como todos han podido comprobar, el gran ""efec..."
3,You have requested a debate on this subject in...,Sus Señorías han solicitado un debate sobre el...
4,"In the meantime, I should like to observe a mi...","A la espera de que se produzca, de acuerdo con..."
...,...,...
524367,Now you must win a majority in this Parliament...,"Ahora bien, usted tiene que conquistar una may..."
524368,I must say to you that we are carrying out an ...,Tengo que decirle que estamos llevando a cabo ...
524369,It remains to be seen whether you deserve it o...,Habrá que ver si la merece y la consigue.\n
524370,But please allow me to express some of the res...,Pero permítame que le exprese alguna de las re...


Construimos un dataset partiendo de nuestro fichero csv

In [ ]:
from datasets import load_dataset

In [ ]:
texts_en_es_dataset = load_dataset("csv", data_files="texts_en_es.csv")
texts_en_es_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e28d836cf7637997/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['EN', 'ES'],
        num_rows: 524372
    })
})

El dataset es demasiado grande y me tomaría mucho tiempo procesarlo con tantos rows, así que voy a dividirlo y coger un subconjunto del mismo para usarlo como mi dataset para la práctica:

In [ ]:
split_datasets = texts_en_es_dataset["train"].train_test_split(train_size=0.98, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['EN', 'ES'],
        num_rows: 513884
    })
    test: Dataset({
        features: ['EN', 'ES'],
        num_rows: 10488
    })
})

In [ ]:
split_datasets["newtrain"] = split_datasets.pop("test")
split_datasets

DatasetDict({
    train: Dataset({
        features: ['EN', 'ES'],
        num_rows: 513884
    })
    newtrain: Dataset({
        features: ['EN', 'ES'],
        num_rows: 10488
    })
})

In [ ]:
new_split_datasets = split_datasets["newtrain"].train_test_split(train_size=0.9, seed=20)
new_split_datasets

DatasetDict({
    train: Dataset({
        features: ['EN', 'ES'],
        num_rows: 9439
    })
    test: Dataset({
        features: ['EN', 'ES'],
        num_rows: 1049
    })
})

Listo para subirlo:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Ahora ya podemos subir nuestro dataset con el método `push_to_hub` indicando el nombre del dataset.

In [ ]:
new_split_datasets.push_to_hub("corpus-en-es")


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Ahora está disponible en: https://huggingface.co/datasets/leorena/corpus-en-es